Purpose of this notebook was a (hopefully) one-time use to add/update product H after the enumaeration of that had benn corrected

In [5]:
import pickle as pkl
import json
import sqlite3 as sql
from pathlib import Path
import pandas as pd
from rdkit.Chem import Draw
from rdkit.Chem.rdMolDescriptors import CalcMolFormula, CalcExactMolWt
from rdkit.Chem.rdmolfiles import ForwardSDMolSupplier
from rdkit import Chem

import gzip
from util_functions import *
from rdkit import Chem
from rdkit.Chem import Draw, SaltRemover, AllChem
from rdkit.Chem.SimpleEnum.Enumerator import EnumerateReaction

import gzip
import warnings
import string
import itertools




In [2]:
DATA_DIR = Path('..', 'data').resolve()
INFO_PATH = DATA_DIR / 'library_static' / 'library_constituents_dataframe.pkl'
MAPPING_PATH = DATA_DIR / 'buildingblocks' / 'compound_mapping.txt'
EXP_PLAN_PATH = DATA_DIR / 'library_info' / 'synthesis_plan.json'
DB_PATH = DATA_DIR / 'db' / '50k_project.db'
STATIC_DIR = DATA_DIR / 'db' / 'static'

In [3]:
# open the sdf.gz
product_dicts = {}
# for product_type in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
for product_type in ['H']:  # I need this to update products H. The commented version is the original one.
    counter = 0
    product_dict = {}
    with gzip.open(DATA_DIR / 'library_static' / f'product_{product_type}.sdf.gz') as file:
        supplier = ForwardSDMolSupplier(file)
        for i, mol in enumerate(supplier):
            if mol is not None:

                smiles = Chem.MolToSmiles(mol)
                molecular_formula_1 = CalcMolFormula(mol)
                mass_1 = CalcExactMolWt(mol)

                product_dict[i] = (smiles, molecular_formula_1, mass_1)

                # # now we need to match the mol with a database entry
                # result = cur.execute('SELECT id FROM virtuallibrary WHERE long_name = ? AND type = ?;', [name, product_type]).fetchall()
                # # check if we got exactly one id back
                # try:
                #     if len(result) == 0:
                #         raise ValueError(f'No entry in DB for long name {name}')
                #     elif len(result) > 1:
                #         raise ValueError(f'Duplicate entry for long name {name} and type {product_type}. Offending entries: {result}')
                #     # sine we now have the correct id, we can add the new info obtained from the sdf file
                #     cur.execute('UPDATE virtuallibrary SET SMILES = ?, molecular_formula_1 = ?, lcms_mass_1 = ?  WHERE id = ?', (smiles, molecular_formula_1, mass_1, result[0][0]))
                # except ValueError:
                #     # only raise the error if the building block was used in 50k project
                #     if name.split(' + ')[0] in compound_long_names and name.split(' + ')[1] in compound_long_names and name.split(' + ')[2] in compound_long_names:
                #         raise ValueError(f'Failed on product {name}')
    product_dicts[product_type] = product_dict
    print(f'{product_type} done')

H done


In [6]:
# we need to infer names

compounds = pd.read_pickle(DATA_DIR / 'library_info' / 'library_constituents_dataframe.pkl')
remover = SaltRemover.SaltRemover()
compounds['desalted'] = compounds.loc[:, 'mol'].apply(remover.StripMol)

compounds['desalted_SMILES'] = compounds.loc[:, 'desalted'].apply(Chem.MolToSmiles)
all_KAT = compounds[compounds.loc[:, 'Category'].str.startswith('I')]
all_Mon = compounds[compounds.loc[:, 'Category'].str.startswith('M')]
all_Spiro = compounds[compounds.loc[:, 'Compound Name'].str.startswith('Spiro')]
all_Fused = compounds[compounds.loc[:, 'Compound Name'].str.startswith('Fused')]
all_Sub = compounds[compounds.loc[:, 'Compound Name'].str.startswith('Mon')]
all_TerTH = compounds[compounds.loc[:, 'Compound Name'].str.startswith('TerTH')]
all_TerABT = compounds[compounds.loc[:, 'Compound Name'].str.startswith('TerABT')]

I = all_KAT['desalted'].tolist()
M = all_Mon['desalted'].tolist()
fused = all_Fused['desalted'].tolist()
spiro = all_Spiro['desalted'].tolist()
sub = all_Sub['desalted'].tolist()
T_TH = all_TerTH['desalted'].tolist()
T_ABT = all_TerABT['desalted'].tolist()
reactant_I = I
reactant_M = M
reactant_ABT = T_ABT
reactant_TH = T_TH

In [7]:
names = [f'{i.GetProp("_Name")} + {m.GetProp("_Name")} + {t.GetProp("_Name")}'
             for i in reactant_I
             for m in reactant_M
             for t in reactant_ABT
             ]  # generate the names from reactant properties

names2 = [f'{i.GetProp("_Name")} + {m.GetProp("_Name")} + {t.GetProp("_Name")}'
             for i in reactant_I
             for m in reactant_M
             for t in reactant_TH
             ]  # generate the names from reactant properties

all_names = names + names2

In [8]:
len(all_names)

236652

In [9]:
len(product_dict)

236652

In [24]:
named_product_dict = {k:v for v, k in zip(product_dict.values(), all_names)}  # assign the name property

In [22]:
# checks
for i, (k, v) in enumerate(named_product_dict.items()):
    if i in [15000, 50000, 34000]:
        print(f'{k}:{v}')

3-Pyr003 + Mon017 + TerABT009:('Cc1ccc2sc(C=CCCC(=O)OC(C)(C)C)nc2c1', 'C17H21NO2S', 303.129299912)
Al002 + Mon082 + TerABT022:('CCOc1ccc2nc(C=CCc3cn(C(=O)OC(C)(C)C)c4ccccc34)sc2c1', 'C25H26N2O3S', 434.1664136919999)
BiPh001 + Mon086 + TerABT017:('Brc1ccc2nc(C=Cc3cccnc3)sc2c1', 'C14H9BrN2S', 315.966981388)


In [25]:
with open(DATA_DIR / 'library_static' / 'H_dict_named.pkl', 'wb') as file:
    pkl.dump(named_product_dict, file)
